# Project I: Multi-Armed Bandits

The multi-armed bandit problem considers a resource allocation problem in partially unknown environments. In more detail, the problem refers to the task of allocating a fixed and limited set of resources among alternative choices in a way that maximizes the expected gain. Importantly, each choice's properties are only partially known at the time of allocation and may become better understood with time and/or by allocating resources to a choice. The multi-armed bandit problem is a classic reinforcement learning problem that demonstrates the exploration-exploitation tradeoff dilemma and related concepts in a clear non-associative form. As such, it helps set the foundations of reinforcement learning theory. The multi-armed bandit problem has also applications in various domains such as clinical trials and networking. In this notebook we will experiment with a number of multiarmed bandit algorithms by altering the abstract bandit algorithm.

![robot](img/robot.png)

## Section 1: General Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Section 2: The General Bandit Algorithm

![algorithm](img/abstract_algorithm.png)

Here we declare a general bandit algorithm that accepts various initialization, action selection and update strategies.

In [ ]:
def bandit(inputs):
    """
        An abstract k-arm bandit algorithm.

        Args:
            inputs: A dictionary containing the number of actions "k", the number of iterations "horizon",
                    the true unknown rewards "q_star", the initialization strategy function "initialization",
                    the action selection strategy function "action_selection", the update rule function "update rule"
                    and any other parameters required.

        Returns:
            q_estimates: list - the estimates of the q values.
            total_reward: float - the cummulative reward
            mean_rewards: list - the consecutive mean rewards
            regret: list - the true maximum regret

    """
    
    #Initializing rewards and regret (used mainly for evaluation purposes)
    total_reward = 0
    mean_rewards = np.zeros(inputs["horizon"])
    regret = np.zeros(inputs["horizon"])
    
    #Initialize an action selection counter (used mainly for the true average update rule)
    action_select_counter = np.zeros(k)
    
    ### INITIALIZE THE ESTIMATES
    q_estimates = inputs["initialization"](inputs["k"], inputs["q_star"])
    
    for t in range(1,inputs["horizon"]+1): # loop over the horizon
        
        ### ACTION SELECTION
        action = inputs["action_selection"](q_estimates, inputs, t, action_select_counter)    
        # Update action counter
        action_select_counter[action] += 1
    
        ## BANDIT CALL
        reward = run_bandit(action, q_star) 
        
        #Calculate reward, mean reward and regret for evaluatuion purposes
        total_reward += reward      
        mean_rewards[t-1] = total_reward/t
        regret[t-1] = np.max(q_star) - reward # We use the true maximum to calculate the regret :)
        
        ## UPDATE ACTION-VALUE ESTIMATES
        q_estimates = inputs["update_rule"](q_estimates, action, reward, inputs, action_select_counter, mean_rewards[t-1])
        
    return q_estimates, total_reward, mean_rewards, regret

In [ ]:
def run_bandit(action, q_star):
    """ 
        Returns the reward of an action assuming that it follows a normal distribution with variance 1.
        The mean values are extracted from q_star 
    """
    return rng2.normal(q_star[action], 1)

### 2.1. Initialization strategies
Using the q* values, we create initialization methods that implement zero, pessimistic, average, optimistic and random estimate initializations for the bandit algorithm:

#### Estimate initialization with zeroes:

In [ ]:
def ZeroInitialization(k, q_star):
    """Zero initialization"""
    return np.zeros(k)

#### Pessimistic estimate initialization:

In [ ]:
def MinInitialization(k, q_star):
    """Min initialization"""
    return np.ones(k) * np.min(q_star)

#### Mean estimate initialization:

In [ ]:
def MeanInitialization(k, q_star):
    """Mean initialization"""
    return np.ones(k) * np.mean(q_star)

#### Optimistic estimate initialization:

In [ ]:
def MaxInitialization(k, q_star):
    """Max initialization"""
    return np.ones(k) * np.max(q_star)

#### Estimate initialization with random numbers:

In [ ]:
def RandomInitialization(k, q_star):
    """Random initialization"""
    return rng3.normal(0, 1, k)  #let's assume we know the distributions and take nice random initial values :)

### 2.2. Action selection strategies

#### Random action selection: 
This strategy selects an action completely at random without taking into account the learned estimates.

In [ ]:
def random_action_selection(estimates, bandit_inputs, t, action_select_counter):
    """Choose an action randomly (uniform)"""
    return rng2.randint(inputs["k"])

#### Greedy action selection:
This action selection strategy always selects the best possible action.

In [ ]:
def greedy_action_selection(estimates, inputs, t, action_select_counter):
    """ Choose an action using the greedy action selection"""
    return np.argmax(estimates)

#### ε-Greedy action selection:
The implementation of the e-Greedy algorithm for the multi-armed bandit problem.

In [ ]:
def e_greedy_action_selection(estimates, inputs, t, action_select_counter):
    """ Choose an action using the e-greedy action selection"""
    # Generate a random number
    p = rng2.rand()
    
    # E-Greedy action selection
    if p < inputs["epsilon"]:
        # Randomly select an action
        return rng2.choice(inputs["k"])
    else:
        # Take greedy action
        return np.argmax(estimates)

#### ε-Greedy with epsion decay action selection:
The implementation of the e-Greedy algorithm with epsilon decay for the multi-armed bandit problem.

In [ ]:
def e_decay_greedy_action_selection(estimates, inputs, t, action_select_counter):
    """ Choose an action using the e-greedy with epsilon decay action selection"""
    # Generate a random number
    p = rng2.rand()

    # E-Greedy action selection
    if p < inputs["epsilon"]*np.exp(-inputs["kappa"]*t):
        # Randomly select an action
        action = rng2.choice(inputs["k"])
    else:
        # Take greedy action
        action = np.argmax(estimates)
            
    return action

#### Upper Confidence Bound action selection:
The implementation of the UCB algorithm for the multi armed bandit problem.

In [ ]:
def ucb_action_selection(estimates, inputs, t, action_select_counter):
    """ Choose an action using the UCB action selection"""
    t = np.sum(action_select_counter) + 1
    # Select action according to UCB Criteria
    return np.argmax(estimates + inputs["c"] * np.sqrt(np.log(t) / (action_select_counter+1)))

#### SoftMax action selection:
The implementation of the softmax bandit algorithm for the multi-armed bandit problem. 

In [ ]:
def softmax_action_selection(estimates, inputs, t, action_select_counter):
    """ Choose an action using the Softmax action selection"""
    # Softmax action selection
    action = rng3.choice(inputs["k"], 1, p=np.exp(estimates/inputs["tau"]) / np.sum(np.exp(estimates/inputs["tau"])))
    return action[0]

### 2.3. Update strategies 

#### True average update rule:

In [ ]:
def update_rule(estimates, action, reward, inputs, action_select_counter, mean_reward):
    """ Return the estimates using the true average update rule"""
    # Update the estimates
    estimates[action] = estimates[action] + (reward - estimates[action]) / action_select_counter[action]
    return estimates

#### Constant learning rate update rule:

In [ ]:
def update_rule_constant_lr(estimates, action, reward, inputs, action_select_counter, mean_reward):
    # Update the estimates
    """ Return the estimates using the constant learning rate update rule"""
    estimates[action] = estimates[action] + (reward - estimates[action]) / (action_select_counter[action]*inputs["a"])
    return estimates

What about decaying learning rate?

## Section 3: The Case Study Experiment

Here we declare the number of trials and arms of the multi-armed bandit problem. We also select a gaussian probability distribution to generate the true values of each arm and to simulate the randomness of the experiment.

In [ ]:
# Number of actions
k = 10
# Number of iterations
horizon = 5000
#Number of trials
trials=100

# Initialization of the true action values (q*) according to a gaussian distribution

q_star = [-0.70318731, -0.49028236, -0.32181433, -1.75507872,  0.20666447, -2.01126457, -0.55725071,  0.33721701,  1.54883597, -1.37073656]
#rng1 = np.random.RandomState(1337) #https://en.wikipedia.org/wiki/Leet
#q_star = rng1.normal(0, 1, k)
#print(q_star)

### 3.1. Experimental procedure 

Define the experimental procedure followed for all experiments

In [ ]:
def procedure(inputs, N):
    """ 
        Does "trials" amount of runs of the algorithm specified by the "inputs" dictionary. 
    
        Args:
            inputs: A dictionary containing the number of actions "k", the number of iterations "horizon",
                    the true unknown rewards "q_star", the initialization strategy function "initialization",
                    the action selection strategy function "action_selection", the update rule function "update rule"
                    and any other parameters required.
            trials: The amount of trial runs to execute so we can get the average performance of the algorithm

        Returns:
            average_q_estimates: list - the average estimates of the q values.
            average_total_reward: float - the average cummulative reward
            average_mean_rewards: list - the average consecutive mean rewards
            average_regret: list - the average true maximum regret
    """
    #Initialize the evaluation parameters
    estimates = np.zeros(k)
    total_reward = 0
    mean_rewards = np.zeros(N)
    regret = np.zeros(N)

    #Run the n trials of the experiment
    for i in range(trials):
        i_estimates, i_total_reward, i_mean_rewards, i_regret = bandit(inputs)
        estimates += i_estimates
        total_reward += i_total_reward
        mean_rewards += i_mean_rewards
        regret += i_regret
        
    return estimates/trials, total_reward/trials, mean_rewards/trials, regret/trials #return the average results for all trials

### 3.2. Plotting/printing:
After each experiment we print two plots to evaluate the performance of the algorithm. We print the loss that we incur due to time/rounds spent due to the learning, or else the regret, and the expected reward of the algorithm across the rounds of each experiment.

Definition of the print and plot of the results (we use this throughout the rest of the notebook):

In [ ]:
def plot_bandits(N, estimates, total_reward, mean_rewards, regret, q_star):
    """ Plots using matplotlib the regret curve and the average reward curve"""
    print("True q values:{}".format(q_star))
    print("Learned Estimates: {}".format(estimates))
    print("")
    print("Euclidean distance from q_star vector: {}".format(np.linalg.norm(q_star-estimates)))
    print("Total Reward: {}".format(total_reward))
    print("Mean Reward: {}".format(mean_rewards[-1]))

    fig, ax = plt.subplots()
    ax.plot(np.linspace(0,N-1,N), np.cumsum(regret), 'b-')
    ax.set_xlabel("Time")
    ax.set_ylabel("Speed")
    ax.grid()
    ax.set(xlabel='Time Steps', ylabel='Total Regret',
           title='Regret Curve')

    plt.show()

    fig, ax = plt.subplots()
    ax.plot(np.linspace(0,N-1,N), mean_rewards, 'b-')
    ax.set_xlabel("Time")
    ax.set_ylabel("Speed")
    ax.grid()
    ax.set(xlabel='Time Steps', ylabel='Average Reward',
           title='Average Reward Curve')

    plt.show()

Definition of the comp_plot, that plots the comparative plot of the various initializations:

In [ ]:
def comp_plot(regret, pes_regret, avg_regret, opt_regret, rnd_regret, \
              mean_rewards, pes_mean_rewards, avg_mean_rewards, opt_mean_rewards, rnd_mean_rewards):
    """ Plots using matplotlib a comparison of the the regret curves and the average reward curves"""
    fig, ax = plt.subplots()
    ax.plot(np.linspace(0,horizon-1,horizon), np.cumsum(regret), 'b-')
    ax.plot(np.linspace(0,horizon-1,horizon), np.cumsum(opt_regret), 'g-')
    ax.plot(np.linspace(0,horizon-1,horizon), np.cumsum(avg_regret), 'r-')
    ax.plot(np.linspace(0,horizon-1,horizon), np.cumsum(pes_regret), 'c-')
    ax.plot(np.linspace(0,horizon-1,horizon), np.cumsum(rnd_regret), 'y-')
    ax.set_xlabel("Time")
    ax.set_ylabel("Speed")
    ax.grid()
    ax.set(xlabel='Time Steps', ylabel='Total Regret',
           title='Regret Curve')
    plt.legend(['Zero','Optimistic','Mean','Pesimistic','Random'])
    plt.show()

    fig, ax = plt.subplots()
    ax.plot(np.linspace(0,horizon-1,horizon), mean_rewards, 'b-')
    ax.plot(np.linspace(0,horizon-1,horizon), opt_mean_rewards, 'g-')
    ax.plot(np.linspace(0,horizon-1,horizon), avg_mean_rewards, 'r-')
    ax.plot(np.linspace(0,horizon-1,horizon), pes_mean_rewards, 'c-')
    ax.plot(np.linspace(0,horizon-1,horizon), rnd_mean_rewards, 'y-')
    ax.set_xlabel("Time")
    ax.set_ylabel("Speed")
    ax.grid()
    ax.set(xlabel='Time Steps', ylabel='Average Reward',
           title='Average Reward Curve')
    plt.legend(['Zero','Optimistic','Mean','Pesimistic','Random'])
    plt.show()

## Section 4. The bandit algorithm with random action selection and true average update (Experiment I)

### 4.1. Initializing with zero values:
What do you think the expected reward is? 

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : ZeroInitialization, # Initialization strategy
          "action_selection"  : random_action_selection, # Action selection strategy
          "update_rule"       : update_rule # The update rule
         }

#Run the experiment and plot:
estimates_rand, total_reward_rand, mean_rewards_rand, regret_rand = procedure(inputs,horizon)
plot_bandits(horizon, estimates_rand, total_reward_rand, mean_rewards_rand, regret_rand, q_star)

### 4.2. Initializing with Pessimistic:
What do you think the expected reward is? 

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MinInitialization, # Initialization strategy
          "action_selection"  : random_action_selection, # Action selection strategy
          "update_rule"       : update_rule # The update rule
         }

#Run the experiment and plot:
min_estimates_rand, min_total_reward_rand, min_mean_rewards_rand, min_regret_rand = procedure(inputs,horizon)
plot_bandits(horizon, min_estimates_rand, min_total_reward_rand, min_mean_rewards_rand, min_regret_rand, q_star)

### 4.3. Initializing with Mean:
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MeanInitialization, # Initialization strategy
          "action_selection"  : random_action_selection, # Action selection strategy
          "update_rule"       : update_rule # The update rule
         }

#Run the experiment and plot:
avg_estimates_rand, avg_total_reward_rand, avg_mean_rewards_rand, avg_regret_rand = procedure(inputs,horizon)
plot_bandits(horizon, avg_estimates_rand, avg_total_reward_rand, avg_mean_rewards_rand, avg_regret_rand, q_star)

### 4.4. Initializing with Optimistic:
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MaxInitialization, # Initialization strategy
          "action_selection"  : random_action_selection, # Action selection strategy
          "update_rule"       : update_rule # The update rule
         }

#Run the experiment and plot:
max_estimates_rand, max_total_reward_rand, max_mean_rewards_rand, max_regret_rand = procedure(inputs,horizon)
plot_bandits(horizon, max_estimates_rand, max_total_reward_rand, max_mean_rewards_rand, max_regret_rand, q_star)

### 4.5. Initializing with Random
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : RandomInitialization, # Initialization strategy
          "action_selection"  : random_action_selection, # Action selection strategy
          "update_rule"       : update_rule # The update rule
         }

#Run the experiment and plot:
rnd_estimates_rand, rnd_total_reward_rand, rnd_mean_rewards_rand, rnd_regret_rand = procedure(inputs,horizon)
plot_bandits(horizon, rnd_estimates_rand, rnd_total_reward_rand, rnd_mean_rewards_rand, rnd_regret_rand, q_star)

### 4.6. Compare all:

Let's see a comparison between the different initialization strategies for the random action selection strategy:

In [ ]:
comp_plot(regret_rand, min_regret_rand, avg_regret_rand, max_regret_rand, rnd_regret_rand, \
          mean_rewards_rand, min_mean_rewards_rand, avg_mean_rewards_rand, max_mean_rewards_rand, rnd_mean_rewards_rand)

## Section 5: The bandit algorithm with greedy action selection and true average update (Experiment II)

### 5.1. Initializing with zero values:
What do you think the expected reward is? 

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : ZeroInitialization, # Initialization strategy
          "action_selection"  : greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule # The update rule
         }

#Run the experiment and plot:
estimates_greedy, total_reward_greedy, mean_rewards_greedy, regret_greedy = procedure(inputs,horizon)
plot_bandits(horizon, estimates_greedy, total_reward_greedy, mean_rewards_greedy, regret_greedy, q_star)

### 5.2. Initializing with Pessimistic:
What do you think the expected reward is? 

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MinInitialization, # Initialization strategy
          "action_selection"  : greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule # The update rule
         }

#Run the experiment and plot:
min_estimates_greedy, min_total_reward_greedy, min_mean_rewards_greedy, min_regret_greedy = procedure(inputs,horizon)
plot_bandits(horizon, min_estimates_greedy, min_total_reward_greedy, min_mean_rewards_greedy, min_regret_greedy, q_star)

### 5.3. Initializing with Mean:
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MeanInitialization, # Initialization strategy
          "action_selection"  : greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule # The update rule
         }

#Run the experiment and plot:
avg_estimates_greedy, avg_total_reward_greedy, avg_mean_rewards_greedy, avg_regret_greedy = procedure(inputs,horizon)
plot_bandits(horizon, avg_estimates_greedy, avg_total_reward_greedy, avg_mean_rewards_greedy, avg_regret_greedy, q_star)

### 5.4. Initializing with Optimistic:
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MaxInitialization, # Initialization strategy
          "action_selection"  : greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule # The update rule
         }

#Run the experiment and plot:
max_estimates_greedy, max_total_reward_greedy, max_mean_rewards_greedy, max_regret_greedy = procedure(inputs,horizon)
plot_bandits(horizon, max_estimates_greedy, max_total_reward_greedy, max_mean_rewards_greedy, max_regret_greedy, q_star)

### 5.5. Initializing with Random
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : RandomInitialization, # Initialization strategy
          "action_selection"  : greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule # The update rule
         }

#Run the experiment and plot:
rnd_estimates_greedy, rnd_total_reward_greedy, rnd_mean_rewards_greedy, rnd_regret_greedy = procedure(inputs,horizon)
plot_bandits(horizon, rnd_estimates_greedy, rnd_total_reward_greedy, rnd_mean_rewards_greedy, rnd_regret_greedy, q_star)

### 5.6 Compare all:

Let's see a comparison between the different initialization strategies for the random action selection strategy:

In [ ]:
comp_plot(regret_greedy, min_regret_greedy, avg_regret_greedy, max_regret_greedy, rnd_regret_greedy, \
          mean_rewards_greedy, min_mean_rewards_greedy, avg_mean_rewards_greedy, max_mean_rewards_greedy, \
          rnd_mean_rewards_greedy)

## Section 6: The bandit algorithm with ε-Greedy action selection and true average update (Experiment III)

#### <div style="text-align: right"><span style="color:red">Maybe test out different ε values? :)</span></div>

### 6.1. Initializing with zero values:
What do you think the expected reward is? 

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : ZeroInitialization, # Initialization strategy
          "action_selection"  : e_greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "epsilon"           : 0.15
         }

#Run the experiment and plot:
estimates_e_greedy, total_reward_e_greedy, mean_rewards_e_greedy, regret_e_greedy = procedure(inputs,horizon)
plot_bandits(horizon, estimates_e_greedy, total_reward_e_greedy, mean_rewards_e_greedy, regret_e_greedy, q_star)

### 6.2. Initializing with Pessimistic:
What do you think the expected reward is? 

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MinInitialization, # Initialization strategy
          "action_selection"  : e_greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "epsilon"           : 0.15
         }

#Run the experiment and plot:
min_estimates_e_greedy, min_total_reward_e_greedy, min_mean_rewards_e_greedy, min_regret_e_greedy = procedure(inputs,horizon)
plot_bandits(horizon, min_estimates_e_greedy, min_total_reward_e_greedy, min_mean_rewards_e_greedy, min_regret_e_greedy, q_star)

### 6.3. Initializing with Mean:
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MeanInitialization, # Initialization strategy
          "action_selection"  : e_greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "epsilon"           : 0.15
         }

#Run the experiment and plot:
avg_estimates_e_greedy, avg_total_reward_e_greedy, avg_mean_rewards_e_greedy, avg_regret_e_greedy = procedure(inputs,horizon)
plot_bandits(horizon, avg_estimates_e_greedy, avg_total_reward_e_greedy, avg_mean_rewards_e_greedy, avg_regret_e_greedy, q_star)

### 6.4. Initializing with Optimistic:
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MaxInitialization, # Initialization strategy
          "action_selection"  : e_greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "epsilon"           : 0.15
         }

#Run the experiment and plot:
max_estimates_e_greedy, max_total_reward_e_greedy, max_mean_rewards_e_greedy, max_regret_e_greedy = procedure(inputs,horizon)
plot_bandits(horizon, max_estimates_e_greedy, max_total_reward_e_greedy, max_mean_rewards_e_greedy, max_regret_e_greedy, q_star)

### 6.5. Initializing with Random
What do you think the expected reward is?
(we will use the average of 100 random initializations, so that the results are significant):

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : RandomInitialization, # Initialization strategy
          "action_selection"  : e_greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "epsilon"           : 0.15
         }

#Run the experiment and plot:
rnd_estimates_e_greedy, rnd_total_reward_e_greedy, rnd_mean_rewards_e_greedy, rnd_regret_e_greedy = procedure(inputs,horizon)
plot_bandits(horizon, rnd_estimates_e_greedy, rnd_total_reward_e_greedy, rnd_mean_rewards_e_greedy, rnd_regret_e_greedy, q_star)

### 6.6. Compare all:

Let's see a comparison between the different initialization strategies for the e-greedy action selection strategy:

In [ ]:
comp_plot(regret_e_greedy, min_regret_e_greedy, avg_regret_e_greedy, max_regret_e_greedy, rnd_regret_e_greedy, \
          mean_rewards_e_greedy, min_mean_rewards_e_greedy, avg_mean_rewards_e_greedy, max_mean_rewards_e_greedy, \
          rnd_mean_rewards_e_greedy)

## Section 7: The bandit algorithm with ε-Greedy with epsilon decay action selection and true average update (Experiment IV)

#### <div style="text-align: right"><span style="color:red">Maybe test out different ε and kappa values? (Especially the limit cases 0, 1):)</span></div>

### 7.1. Initializing with zero values:
What do you think the expected reward is? 

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : ZeroInitialization, # Initialization strategy
          "action_selection"  : e_decay_greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "epsilon"           : 0.1,# The epsilon parameter. (try with 0.01 as well--will it get stuck to lockal minima?)
          "kappa"             : 0.001, # Decay coefficient
         }

#Run the experiment and plot:
estimates_e_decay_greedy, total_reward_e_decay_greedy, mean_rewards_e_decay_greedy, regret_e_decay_greedy = procedure(inputs,horizon)
plot_bandits(horizon, estimates_e_decay_greedy, total_reward_e_decay_greedy, mean_rewards_e_decay_greedy, regret_e_decay_greedy, q_star)

### 7.2. Initializing with Pessimistic:
What do you think the expected reward is? 

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MinInitialization, # Initialization strategy
          "action_selection"  : e_decay_greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "epsilon"           : 0.1,# The epsilon parameter. (try with 0.01 as well--will it get stuck to lockal minima?)
          "kappa"             : 0.001, # Decay coefficient
         }

#Run the experiment and plot:
min_estimates_e_decay_greedy, min_total_reward_e_decay_greedy, min_mean_rewards_e_decay_greedy, min_regret_e_decay_greedy = procedure(inputs,horizon)
plot_bandits(horizon, min_estimates_e_decay_greedy, min_total_reward_e_decay_greedy, min_mean_rewards_e_decay_greedy, min_regret_e_decay_greedy, q_star)

### 7.3. Initializing with Mean:
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MeanInitialization, # Initialization strategy
          "action_selection"  : e_decay_greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "epsilon"           : 0.1,# The epsilon parameter. (try with 0.01 as well--will it get stuck to lockal minima?)
          "kappa"             : 0.001, # Decay coefficient
         }

#Run the experiment and plot:
avg_estimates_e_decay_greedy, avg_total_reward_e_decay_greedy, avg_mean_rewards_e_decay_greedy, avg_regret_e_decay_greedy = procedure(inputs,horizon)
plot_bandits(horizon, avg_estimates_e_decay_greedy, avg_total_reward_e_decay_greedy, avg_mean_rewards_e_decay_greedy, avg_regret_e_decay_greedy, q_star)

### 7.4. Initializing with Optimistic:
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MaxInitialization, # Initialization strategy
          "action_selection"  : e_decay_greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "epsilon"           : 0.1,# The epsilon parameter. (try with 0.01 as well--will it get stuck to lockal minima?)
          "kappa"             : 0.001, # Decay coefficient
         }

#Run the experiment and plot:
max_estimates_e_decay_greedy, max_total_reward_e_decay_greedy, max_mean_rewards_e_decay_greedy, max_regret_e_decay_greedy = procedure(inputs,horizon)
plot_bandits(horizon, max_estimates_e_decay_greedy, max_total_reward_e_decay_greedy, max_mean_rewards_e_decay_greedy, max_regret_e_decay_greedy, q_star)

### 7.5. Initializing with Random
What do you think the expected reward is?
(we will use the average of 100 random initializations, so that the results are significant):

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : RandomInitialization, # Initialization strategy
          "action_selection"  : e_decay_greedy_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "epsilon"           : 0.1,# The epsilon parameter. (try with 0.01 as well--will it get stuck to lockal minima?)
          "kappa"             : 0.001, # Decay coefficient
         }

#Run the experiment and plot:
rnd_estimates_e_decay_greedy, rnd_total_reward_e_decay_greedy, rnd_mean_rewards_e_decay_greedy, rnd_regret_e_decay_greedy = procedure(inputs,horizon)
plot_bandits(horizon, rnd_estimates_e_decay_greedy, rnd_total_reward_e_decay_greedy, rnd_mean_rewards_e_decay_greedy, rnd_regret_e_decay_greedy, q_star)

### 7.6. Compare all:

Let's see a comparison between the different initialization strategies for the e-Greedy with epsilon decay action selection strategy:

In [ ]:
comp_plot(regret_e_decay_greedy, min_regret_e_decay_greedy, avg_regret_e_decay_greedy,\
          max_regret_e_decay_greedy, rnd_regret_e_decay_greedy, mean_rewards_e_decay_greedy,\
          min_mean_rewards_e_decay_greedy, avg_mean_rewards_e_decay_greedy,\
          max_mean_rewards_e_decay_greedy, rnd_mean_rewards_e_decay_greedy)

## Section 8: The bandit algorithm with Upper Confidence Bound action selection and true average update (Experiment V)

#### <div style="text-align: right"><span style="color:red">Maybe test out different c values? :)</span></div>

### 8.1. Initializing with zero values:
What do you think the expected reward is? 

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : ZeroInitialization, # Initialization strategy
          "action_selection"  : ucb_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "c"                 : 2. # The c parameter
         }

#Run the experiment and plot:
estimates_ucb, total_reward_ucb, mean_rewards_ucb, regret_ucb = procedure(inputs,horizon)
plot_bandits(horizon, estimates_ucb, total_reward_ucb, mean_rewards_ucb, regret_ucb, q_star)

### 8.2. Initializing with Pessimistic:
What do you think the expected reward is? 

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MinInitialization, # Initialization strategy
          "action_selection"  : ucb_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "c"                 : 2. # The c parameter
         }

#Run the experiment and plot:
min_estimates_ucb, min_total_reward_ucb, min_mean_rewards_ucb, min_regret_ucb = procedure(inputs,horizon)
plot_bandits(horizon, min_estimates_ucb, min_total_reward_ucb, min_mean_rewards_ucb, min_regret_ucb, q_star)

### 8.3. Initializing with Mean:
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MeanInitialization, # Initialization strategy
          "action_selection"  : ucb_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "c"                 : 2. # The c parameter
         }

#Run the experiment and plot:
avg_estimates_ucb, avg_total_reward_ucb, avg_mean_rewards_ucb, avg_regret_ucb = procedure(inputs,horizon)
plot_bandits(horizon, avg_estimates_ucb, avg_total_reward_ucb, avg_mean_rewards_ucb, avg_regret_ucb, q_star)

### 8.4. Initializing with Optimistic:
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MaxInitialization, # Initialization strategy
          "action_selection"  : ucb_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "c"                 : 2. # The c parameter
         }

#Run the experiment and plot:
max_estimates_ucb, max_total_reward_ucb, max_mean_rewards_ucb, max_regret_ucb = procedure(inputs,horizon)
plot_bandits(horizon, max_estimates_ucb, max_total_reward_ucb, max_mean_rewards_ucb, max_regret_ucb, q_star)

### 8.5. Initializing with Random
What do you think the expected reward is?
(we will use the average of 100 random initializations, so that the results are significant):

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : RandomInitialization, # Initialization strategy
          "action_selection"  : ucb_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "c"                 : 2. # The c parameter
         }

#Run the experiment and plot:
rnd_estimates_ucb, rnd_total_reward_ucb, rnd_mean_rewards_ucb, rnd_regret_ucb = procedure(inputs,horizon)
plot_bandits(horizon, rnd_estimates_ucb, rnd_total_reward_ucb, rnd_mean_rewards_ucb, rnd_regret_ucb, q_star)

### 8.6. Compare all:

Let's see a comparison between the different initialization strategies for the ucb action selection strategy:

In [ ]:
comp_plot(regret_ucb, min_regret_ucb, avg_regret_ucb, max_regret_ucb, rnd_regret_ucb, \
          mean_rewards_ucb, min_mean_rewards_ucb, avg_mean_rewards_ucb, max_mean_rewards_ucb, \
          rnd_mean_rewards_ucb)

## Section 9: The bandit algorithm with Softmax action selection and true average update (Experiment VI)

#### <div style="text-align: right"><span style="color:red">Maybe test out different τ values? What happens very close to 0 and with very high τ values? :)</span></div>

### 9.1. Initializing with zero values:
What do you think the expected reward is? 

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : ZeroInitialization, # Initialization strategy
          "action_selection"  : softmax_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "tau"               : 0.3 # The tau parameter
         }

#Run the experiment and plot:
estimates_sm, total_reward_sm, mean_rewards_sm, regret_sm = procedure(inputs,horizon)
plot_bandits(horizon, estimates_sm, total_reward_sm, mean_rewards_sm, regret_sm, q_star)

### 9.2. Initializing with Pessimistic:
What do you think the expected reward is? 

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MinInitialization, # Initialization strategy
          "action_selection"  : softmax_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "tau"               : 0.3 # The tau parameter
         }

#Run the experiment and plot:
min_estimates_sm, min_total_reward_sm, min_mean_rewards_sm, min_regret_sm = procedure(inputs,horizon)
plot_bandits(horizon, min_estimates_sm, min_total_reward_sm, min_mean_rewards_sm, min_regret_sm, q_star)

### 9.3. Initializing with Mean:
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MeanInitialization, # Initialization strategy
          "action_selection"  : softmax_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "tau"               : 0.3 # The tau parameter
         }

#Run the experiment and plot:
avg_estimates_sm, avg_total_reward_sm, avg_mean_rewards_sm, avg_regret_sm = procedure(inputs,horizon)
plot_bandits(horizon, avg_estimates_sm, avg_total_reward_sm, avg_mean_rewards_sm, avg_regret_sm, q_star)

### 9.4. Initializing with Optimistic:
What do you think the expected reward is?

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : MaxInitialization, # Initialization strategy
          "action_selection"  : softmax_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "tau"               : 0.3 # The tau parameter
         }

#Run the experiment and plot:
max_estimates_sm, max_total_reward_sm, max_mean_rewards_sm, max_regret_sm = procedure(inputs,horizon)
plot_bandits(horizon, max_estimates_sm, max_total_reward_sm, max_mean_rewards_sm, max_regret_sm, q_star)

### 9.5. Initializing with Random
What do you think the expected reward is?
(we will use the average of 100 random initializations, so that the results are significant):

In [ ]:
#Seed our random engines:
rng2 = np.random.RandomState(1337)
rng3 = np.random.RandomState(1337)

#Initialize the experiment:
inputs = {"k"                 : k, # Number of actions
          "horizon"           : horizon, # Number of iterations
          "q_star"            : q_star, # The true unknown rewards
          "initialization"    : RandomInitialization, # Initialization strategy
          "action_selection"  : softmax_action_selection, # Action selection strategy
          "update_rule"       : update_rule, # The update rule
          "tau"               : 0.3 # The tau parameter
         }

#Run the experiment and plot:
rnd_estimates_sm, rnd_total_reward_sm, rnd_mean_rewards_sm, rnd_regret_sm = procedure(inputs,horizon)
plot_bandits(horizon, rnd_estimates_sm, rnd_total_reward_sm, rnd_mean_rewards_sm, rnd_regret_sm, q_star)

### 9.6. Compare all:

Let's see a comparison between the different initialization strategies for the softmax action selection strategy:

In [ ]:
comp_plot(regret_sm, min_regret_sm, avg_regret_sm, max_regret_sm, rnd_regret_sm, \
          mean_rewards_sm, min_mean_rewards_sm, avg_mean_rewards_sm, max_mean_rewards_sm, \
          rnd_mean_rewards_sm)

## Section 10: Comparing Bandit Algorithms
Compare the bandit algorithms through plotting.

In [ ]:
### Plot the results of all the algorithms ###
##############################################

fig, ax = plt.subplots()
ax.plot(np.linspace(0,horizon-1,horizon), np.cumsum(regret_rand), 'b-')
ax.plot(np.linspace(0,horizon-1,horizon), np.cumsum(regret_greedy), 'g-')
ax.plot(np.linspace(0,horizon-1,horizon), np.cumsum(regret_e_greedy), 'r-')
ax.plot(np.linspace(0,horizon-1,horizon), np.cumsum(regret_e_decay_greedy), 'c-')
ax.plot(np.linspace(0,horizon-1,horizon), np.cumsum(regret_ucb), 'm-')
ax.plot(np.linspace(0,horizon-1,horizon), np.cumsum(regret_sm), 'y-')
ax.grid()
ax.set(xlabel='Time Steps', ylabel='Total Regret',
       title='Regret Curve')
plt.legend(['Random','Greedy','e-Greedy','e-Greedy with e decay','UCB','Softmax'])
plt.show()

fig, ax = plt.subplots()
ax.plot(np.linspace(0,horizon-1,horizon), mean_rewards_rand, 'b-')
ax.plot(np.linspace(0,horizon-1,horizon), mean_rewards_greedy, 'g-')
ax.plot(np.linspace(0,horizon-1,horizon), mean_rewards_e_greedy, 'r-')
ax.plot(np.linspace(0,horizon-1,horizon), mean_rewards_e_decay_greedy, 'c-')
ax.plot(np.linspace(0,horizon-1,horizon), mean_rewards_ucb, 'm-')
ax.plot(np.linspace(0,horizon-1,horizon), mean_rewards_sm, 'y-')
ax.grid()
ax.set(xlabel='Time Steps', ylabel='Average Reward',
       title='Average Reward Curve')
plt.legend(['Random','Greedy','e-Greedy','e-Greedy with e decay','UCB','Softmax'])
plt.show()

## Section 11: Exercises

### 11.1. Exercise 1: Implement the gradient bandits action selection and update rule methods yourself:

#### Run the gradient bandits with zeros initialization:

#### Will gradient bandits have a different result using other initializations?
Run the gradient bandits using different initializations to find out if you were correct:

### 11.2. Exercise 2: Implement a bandit of your own liking!
(A hybrid bandit maybe? Play with constant step size? Compare different greedy ones? Different gradient-based ones?)
